In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

# Funciones

In [2]:
# linea es un string de la forma:
# "n0, n1, n2, ..., n10, r" donde nX es una propiedad del tráfico y r es
# su clasificación (1 si es botnet, 0 si no lo es)
def formato_inicial(linea):
    aux   = [float(i) for i in linea.split(',')]
    datos = np.array(aux[:-1])
    resul = aux[-1]
    return (datos, resul)

# Devuelve un RDD de los datos del fichero fn. La lectura con Spark
# ya es mucho más rápida que leyendo con numpy.loadtxt()
def readFile(fn):
    global sc
    todo = sc.textFile(fn)
    return todo.map(formato_inicial)

In [4]:
def norm_tupla(tuplaXy):
    media = np.mean(tuplaXy[0])
    stdev = np.std(tuplaXy[0])
    tnorm = (tuplaXy[0] - media)/stdev
    
    return (tnorm, tuplaXy[1])

# Toma por parámetro un RDD de 1.000.000 de registros.
# Cada registro tiene la forma (X, y)
# X es un array con 11 flotantes
# y es 1 ó 0
# Devuelve un RDD equivalente donde la X de cada tupla está
# reescalada a N(0, 1) (media = 0, desv. est. = 1)
def normalizar(rdd):
    return rdd.map(norm_tupla)

In [5]:
def sigmoide(pesos, sesgo, datos):
    pass

def entrenar(datos, iteraciones, aprendizaje, tasareg):
    f = datos.count()            # 1.000.000
    c = len(datos.take(1)[0][0]) # 11
    pesos   = np.array([(i, np.random.random()) for i in range(c)])
    d_pesos = np.array([(i, 0) for i in range(c)])
    sesgo   = 0
    d_sesgo = 0
    for i in range(iteraciones):
        # pred = sigmoide(pesos, sesgo, datos)
        pass
    

# Pruebas

In [6]:
sc  = SparkContext("local[*]", "BotnetParalelaPruebas")
t1  = time.time()
rdd = readFile("../../../datos/botnet_tot_syn_l.csv")
t2  = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 0.41776108741760254 segundos.


In [7]:
datos = normalizar(rdd)
# Haz la media de uno de los X. Tiene que ser ~0
# np.mean(datos.take(1)[0][0]) # 0.000000000000000010092936587501423
# La desviación estándar tiene que ser ~1
# np.std(datos.take(1)[0][0]) # 1.0